In [48]:
!pip install -qU \
pinecone-client==3.1.0\
tenacity==8.2.3




In [13]:
import json
import os

baseFolder = '/Users/shota/github/content-ge/startup-school'
nodesJSONFile = os.path.join(baseFolder, 'nodes.json')
f = open(nodesJSONFile)
nodes = json.load(f)
f.close()


In [46]:
data = []
for key, node in nodes.items():
    textFileName = os.path.join(baseFolder, 'nodes', key, "text1.md")
    with open(textFileName, 'r') as file:
        text = file.read()
        data.append({
            'id': key,
            'name': node['name'],
            'text': text
        })


In [56]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

openai_client = OpenAI(
    api_key = api_key,
    organization=orgID,
)



In [61]:
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model="text-embedding-3-small") -> list[float]:
    response = openai_client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

def get_vector(d):
    emb = get_embedding(d['text'])
    return {
        "id": d['id'],
        "values": emb,
        "metadata": {"name": d['name']}
    }

def create_vector_json(data):
    vectors = []
    for d in data:
        vec = get_vector(d)
        vectors.append(vec)
    with open("vectors.json", "w") as f:
        json.dump(vectors, f)
        
create_vector_json(data)


In [44]:
from pinecone import Pinecone

pinecone_api_key = %env CHAT_GPT_API_KEY
# pinecone_api_key = os.environ.get('PINECONE_API_KEY') or 'xxx'
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'vitsi-ai-first-index'
index = pc.Index(index_name)
index.describe_index_stats()



{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [69]:

f = open("vectors.json")
vectors = json.load(f)
f.close()

# vectors
index.upsert(vectors)





{'upserted_count': 202}